# Monster Hunter Freedom Unite
Uma interface onde o usuário posssa selecionar as informações de Monster Hunter Freedom Unite.
As seguintes informações devem ser acessadas pelo usuário:

    - Lista de Quests e as Key Quests.
        - Elder Quests
        - Nekoht Quests
        - Guild Hall Quests

    - Lista de Monstro:
        - Nome do Monstro em Inglês & Japones.
        - Classe do Monstro
        - Elemento & Fraqueza Elemental
        - Habitat
        - Geração
        - Frase*

    - Lista de Items:
        - Nome, Informação e Preço


In [1]:
import requests
import re

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from pandas import DataFrame

# 1.0. Key Quest List Hall

In [50]:
url = 'https://monsterhunter.fandom.com/wiki/MHFU:_Guild_Quests'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

page = requests.get( url, headers=headers )
soup = BeautifulSoup( page.text, 'html.parser' )

In [51]:
quest_list = [list( filter ( None, p.get_text().split('\n') ) ) for p in soup.find_all( 'table', class_='linetable hover2' )]
quest_list = pd.DataFrame( quest_list ).T

quest_board = [list( filter( None, p.get_text().split('\n') ) ) for p in soup.find_all( 'table', class_='linetable hover2' ) ]

quest_list = pd.DataFrame()

for i in range( len( quest_board ) ):
    quest_index = pd.DataFrame( [list( filter( None, p.get_text().split('\n') ) ) for p in soup.find_all( 'table', class_='linetable hover2' )[i].find_all('tr')] )
    quest_list  = pd.concat( [quest_list, quest_index], axis=0 )

In [52]:
quest_list.columns = ['title', 'goal', 'area', 'time', 'fee', 'reward', 'notes', 'nan', 'nan2']

quest_list = quest_list[~quest_list['title'].str.contains('Quest Title')]

quest_list = quest_list.reset_index()
quest_list = quest_list.drop( columns=['index', 'nan2'] )

In [53]:
df_ref = pd.DataFrame( index=np.arange( len( quest_list ) ) )

u_quests = quest_list[quest_list['title'] == 'URGENT:']
u_quests = u_quests.drop( columns=['title'] )
u_quests.columns = ['title', 'goal', 'area', 'time', 'fee', 'reward', 'notes']

# Title
df_ref = pd.concat( [df_ref, quest_list[~quest_list['title'].str.contains('URGENT')]['title']], axis=1 )
df_ref['title'] = df_ref['title'].combine_first( u_quests['title'] ) 


# goal
quest_goal = quest_list[~quest_list['goal'].str.contains('The Ruler of the Snow')]['goal']
list_remove = quest_list[quest_list['title'] == 'URGENT:']['goal'].tolist()

for i in list_remove:
    quest_goal = quest_goal[~quest_goal.str.contains(i)]
    
df_ref = pd.concat( [df_ref, quest_goal], axis=1)
 
df_ref['goal'] = df_ref['goal'].combine_first( u_quests['goal'] )


# area
quest_area = quest_list[~quest_list['area'].str.contains('Hunt the Blangonga')]['area']
list_remove = quest_list[quest_list['title'] == 'URGENT:']['area'].tolist()

for i in list_remove:
    quest_area = quest_area[~quest_area.str.contains(i)]
    
df_ref = pd.concat( [df_ref, quest_area], axis=1)
 
df_ref['area'] = df_ref['area'].combine_first( u_quests['area'] )


# time
quest_time = quest_list['time'].str.extract('(\d+\w+)')[0]
quest_time.name = 'time'
quest_time = quest_time.combine_first( u_quests['time'] )

df_ref = pd.concat( [df_ref, quest_time], axis=1 )
df_ref.loc[df_ref['time'] == '50', 'time'] = '50mins'


# fee
quest_fee = quest_list[~quest_list['fee'].str.contains('mins')]['fee']
quest_fee = quest_fee.combine_first( u_quests['fee'] )
quest_fee.name = 'fee'

df_ref = pd.concat( [df_ref, quest_fee], axis=1 )


# reward
quest_rewards = quest_list['reward']
quest_new_reward = quest_list[quest_list['title'] == 'URGENT:']['notes'].tolist()
index = [41, 68, 88, 102, 130, 165, 166, 167]

for i in range( len( index ) ):
    quest_rewards[quest_rewards.index[index[i]]] = quest_new_reward[i]
    
quest_rewards.name = 'reward'
df_ref = pd.concat( [df_ref, quest_rewards], axis=1 )


# notes
quest_notes = quest_list[~quest_list['title'].str.contains('URGENT:')]['notes']
quest_notes = quest_notes.combine_first( u_quests['notes'] )
quest_notes.name = 'notes'

df_ref = pd.concat( [df_ref, quest_notes], axis=1 )

df_ref['notes'] = df_ref['notes'].fillna('No Info')

df_ref.to_csv('data/all_quests.csv')

In [124]:
required_quests = df_ref[df_ref['notes'].str.contains('Urgent')]
required_quests = pd.concat( [required_quests, df_ref[df_ref['notes'].str.contains('Unlocks')]], axis=0 )

urgent_quests = df_ref[df_ref['notes'].str.contains('Unlock')]
urgent_quests = pd.concat( [urgent_quests, required_quests[required_quests['notes'].str.contains('Lv')]], axis=0 )

urgent_quests['notes'] = urgent_quests['notes'].str.extract('([a-zA-Z ]+\★+)')
urgent_quests.iloc[ 5, -1] = 'Unlocks the second Urgent "Rise to the Summit"'
urgent_quests.iloc[ 6, -1] = 'Unlocks the third Urgent "Hypno, Hypno!"'

key_quests = required_quests[required_quests['notes'].str.contains('Required')]
key_quests = pd.concat( [key_quests, urgent_quests], axis=0 )
key_quests = key_quests.drop_duplicates()

key_quests = key_quests.reset_index().drop( columns=['index'], axis=1 )
key_quests['title'] = key_quests['title'].apply( lambda x: x.replace('(Key Quest)', ''))

In [125]:
key_quests['rank'] = 1

key_quests.iloc[4:9, -1] = 2
key_quests.iloc[-10, -1] = 2
key_quests.iloc[9:13, -1]  = 3
key_quests.iloc[-9, -1]    = 3
key_quests.iloc[13:17, -1] = 4
key_quests.iloc[-8, -1]    = 4
key_quests.iloc[17:22, -1] = 5
key_quests.iloc[-7, -1]    = 5
key_quests.iloc[22:27, -1] = 6
key_quests.iloc[-6:-3, -1] = 6
key_quests.iloc[27:32, -1] = 7
key_quests.iloc[-3, -1]    = 7
key_quests.iloc[32:37, -1] = 8
key_quests.iloc[-2, -1]    = 8
key_quests.iloc[37:43, -1] = 9
key_quests.iloc[-1, -1]    = 9

key_quests.to_csv('data/key_quests_hall.csv')

In [181]:
df_ref.iloc[-3:, :]

,title,goal,area,time,fee,reward,notes
252,Legendary Black Dragon,Slay the Fatalis,Castle Schrade,50mins,4500z,54000z,Complete at least one weapon training (G-Class...
253,The End Times,Slay the Crimson Fatalis,Battleground,50mins,4500z,54000z,Slay the Fatalis and clear all Elder Dragon qu...
254,Ancestral Dragon,Slay the White Fatalis,Tower,50mins,4500z,54000z,Slay the Crimson Fatalis and clear all G3★ Epi...


# 2.0. Key Quest List Village

## 2.1. Quest Elder

In [2]:
url = 'https://monsterhunter.fandom.com/wiki/MHFU:_Elder_Quests'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

page = requests.get( url, headers=headers )
soup = BeautifulSoup( page.text, 'html.parser' )

In [3]:
quest_board = quest_list = [list( filter ( None, p.get_text().split('\n') ) ) for p in soup.find_all( 'table', class_='linetable hover2' )]
quest_list  = pd.DataFrame( quest_list ).T

quest_list = pd.DataFrame()

for i in range( len( quest_board ) ):
    quest_index = pd.DataFrame( [list( filter( None, p.get_text().split('\n') ) ) for p in soup.find_all( 'table', class_='linetable hover2' )[i].find_all('tr')] )
    quest_list  = pd.concat( [quest_list, quest_index], axis=0 )

quest_list = quest_list.reset_index()
quest_list = quest_list.drop( columns=['index'] )
quest_list.columns = ['title', 'goal', 'area', 'time', 'fee', 'reward', 'notes', 'nan', 'nan2', 'nan3']
quest_list = quest_list[~quest_list['title'].str.contains('Name')]

# Urgent Quest List
urgent = quest_list[quest_list['title'].str.contains('URGENT')].drop( columns=['title', 'nan2', 'nan3'] )
urgent.columns = ['title', 'goal', 'area', 'time', 'fee', 'reward', 'notes']
urgent.iloc[4, -1]  = 'Unlocks LV★★★★★★ quests'
urgent.iloc[5, -1]  = 'Unlocks the second Urgent "Secret Request: Hypnocatrice'
urgent.iloc[-1, -1] = 'Unlocks LV 7★ (Felyne Elder) Quests'

# Quest List
quest_list = quest_list[~quest_list['title'].str.contains('URGENT')]
quest_list = quest_list.reset_index()
quest_list = quest_list.drop( columns=['index'] )

In [117]:
df_ref = pd.DataFrame( index=np.arange( len( quest_list ) ), columns=['title', 'goal', 'area', 'time', 'fee', 'reward', 'required', 'notes'] )

df_ref['title'] = quest_list['title']
df_ref['goal']  = quest_list['goal']
df_ref['area']  = quest_list['area']
df_ref['time']  = quest_list['time']
df_ref['fee']   = quest_list['fee']
df_ref['reward']  = quest_list['reward']

# Required
df_required = quest_list[quest_list['nan'].str.contains('Required', na=False)]['nan']
df_required.name = 'required'
df_required = df_required.combine_first( quest_list[quest_list['notes'].str.contains('Required')]['notes'] )

df_ref['required'] = df_required

# Notes
quest_notes = quest_list[quest_list['notes'].str.contains('Clear')]['notes']
quest_notes.name = 'notes'
quest_notes = quest_notes.combine_first( quest_list[quest_list['nan'].str.contains('Clear', na=False)]['nan'] )
quest_notes = quest_notes.combine_first(quest_list[quest_list['notes'].str.contains('Beat')]['notes'])
df_ref['notes'] = quest_notes

# Cleaning
df_ref['title']    = df_ref['title'].str.extract('([a-zA-Z ]+)').apply( lambda x: x.replace( 'Key Quest', '' ) )[0]
df_ref['required'] = df_ref['required'].str.extract( '([a-zA-Z ]+)' ).apply( lambda x: x.replace('MHFU ', 'Required for next Urgent'), axis=1 )[0]
df_ref['required'] = df_ref['required'].fillna( 'Not Required' )
df_ref['notes'] = df_ref['notes'].fillna( 'No Info' )

df_ref = pd.concat( [df_ref, urgent], axis=0 )
df_ref['required'] = df_ref['required'].fillna( 'Urgent' )

df_ref.to_csv('data/all_quests_elder.csv')

In [210]:
key_quests = df_ref[~df_ref['required'].str.contains('Not')]
key_quests['star'] = 1

key_quests.iloc[3:7, -1]   = 2
key_quests.iloc[-6 , -1]   = 2
key_quests.iloc[7:11, -1]  = 3
key_quests.iloc[-5 , -1]   = 3
key_quests.iloc[11:15, -1] = 4
key_quests.iloc[-4 , -1]   = 4
key_quests.iloc[15:19, -1] = 5
key_quests.iloc[-3 , -1]   = 5
key_quests.iloc[19:22, -1] = 6
key_quests.iloc[-2: , -1]  = 6

key_quests.to_csv('data/key_quests_elder.csv')

<ipython-input-210-f33ea71ffb84>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key_quests['star'] = 1
c:\users\t-gamer\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


# 3.0. Monster List

In [223]:
url = 'https://monsterhunter.fandom.com/wiki/MHFU:_Monsters'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

page = requests.get( url, headers=headers )
soup = BeautifulSoup( page.text, 'html.parser' )

In [224]:
monster_list  = [list( filter( None, p.get_text().split('\n') ) ) for p in soup.find('div', class_='mw-parser-output').find_all('tr')]
monster_names = []

for i in monster_list:
    for j in range( len( i ) ):
        monster_names.append( i[j] )
        
monster_names = [q.replace(' ', '_') for q in [re.match( '[a-zA-Z ]+', q ).group(0) for q in monster_names if re.match( '[a-zA-Z ]+', q ) != None]]
monster_names = monster_names[:-6]

monster_names[29] = 'Yian_Kut-Ku'
monster_names[30] = 'Blue_Yian_Kut-Ku'
monster_names[-6] = 'Lao-Shan_Lung'
monster_names[-5] = 'Ashen_Lao-Shan_Lung'

In [225]:
url = 'https://monsterhunter.fandom.com/wiki/Teostra'
page = requests.get( url, headers=headers )
soup = BeautifulSoup( page.text, 'html.parser' )

monster_info = soup.find_all( 'div', class_='pi-data-value pi-font' )

# Name
monster_jp_name = soup.find( 'ruby' ).get_text()

# Class
info = [p.find('a') for p in monster_info]
monster_class = [p.get_text() for p in info if p != None][0]

# Element
info = soup.find_all( 'small' )
monster_element = [p.get_text().strip() for p in info][0:2]
monster_element = monster_element[0] + ', ' + monster_element[1]

# Weakness
info = soup.find_all( 'small' )
monster_weak = [p.get_text().strip() for p in info]
two_weakness = [re.match( '[a-zA-Z ]+', p ).group(0) for p in monster_weak if re.match( '[a-zA-Z ]+', p ) != None][-2:]
two_weakness = two_weakness[0] + ', ' + two_weakness[1]

# Generation
monster_gen  = monster_info[-1].get_text()

pd.DataFrame( [monster_jp_name, monster_class, monster_element, two_weakness, monster_gen] ).T

,0,1,2,3,4
0,テオ・テスカトル,Elder Dragon,"Dragon, Fire","Water, Ice",Second


In [226]:
df_monster = pd.DataFrame()

for j in ['https://monsterhunter.fandom.com/wiki/'+ i for i in monster_names]:
    page = requests.get( j, headers=headers )
    soup = BeautifulSoup( page.text, 'html.parser' )
    
    monster_info = soup.find_all( 'div', class_='pi-data-value pi-font' )

    # Name
    monster_jp_name = soup.find( 'ruby' ).get_text()

    # Class
    info = [p.find('a') for p in monster_info]
    monster_class = [p.get_text() for p in info if p != None][0]

    # Element
    info = soup.find_all( 'small' )
    monster_element = [p.get_text().strip() for p in info][0:2]
    
    if len(monster_element) > 1:
        monster_element = monster_element[0] + ', ' + monster_element[1]
        
    if len(monster_element) == 0:
        monster_element = None
        
    else:
        monster_element = monster_element

    # Weakness
    info = soup.find_all( 'small' )
    monster_weak = [p.get_text().strip() for p in info]
    two_weakness = [re.match( '[a-zA-Z ]+', p ).group(0) for p in monster_weak if re.match( '[a-zA-Z ]+', p ) != None][-2:]
    
    if len(two_weakness) > 1:
        two_weakness = two_weakness[0] + ', ' + two_weakness[1]
        
    if len(two_weakness) == 0:
        two_weakness = None
        
    else:
        two_weakness = two_weakness

    # Generation
    monster_gen  = monster_info[-1].get_text()
    
    df_info    = pd.DataFrame( [monster_jp_name, monster_class, monster_element, two_weakness, monster_gen] ).T
    df_monster = pd.concat( [df_monster, df_info], axis=0 )

df_monster.columns=['jp_name', 'class', 'element', 'weak', 'generation']

# Element
df_monster['element'] = df_monster['element'].str.extract('([a-zA-Z ]+\,\ [a-zA-Z ]+)')[0]

# Weak
df_monster['weak'] = df_monster['weak'].str.extract('([a-zA-Z ]+\,\ [a-zA-Z ]+)')[0]

# Join Names
df_monster = df_monster.reset_index().drop( columns=['index'], axis=1 )
df_monster = pd.concat( [df_monster, pd.Series( [p.replace('_', ' ') for p in monster_names] ) ], axis=1 )
df_monster.columns=['jp_name', 'class', 'element', 'weak', 'generation', 'name']

df_monster['weak'][7]  = 'Fire'
df_monster['weak'][27] = 'Ice'
df_monster['weak'][49] = 'Ice'
df_monster['weak'][48] = 'Ice'
df_monster['weak'][70] = 'Fire, Water'
df_monster['weak'] = df_monster['weak'].fillna('Thunder')

df_monster['element'] = df_monster['element'].fillna( 'None' )
df_monster['element'][27] = 'None'
df_monster['element'][21] = 'None'
df_monster.loc[df_monster['element'] == '', 'element'] = 'None'
df_monster.loc[df_monster['element'].str.contains('Thunderblight'), 'element'] = 'Thunder'
df_monster.loc[df_monster['element'].str.contains('Waterblight'), 'element']   = 'Water'
df_monster.loc[df_monster['element'].str.contains('Iceblight'), 'element']     = 'Ice'
df_monster.loc[df_monster['element'].str.contains('Fireblight'), 'element']    = 'Fire'

In [234]:
pd.DataFrame( [df_monster['name'], 
               df_monster['jp_name'], 
               df_monster['class'], 
               df_monster['element'], 
               df_monster['weak'], 
               df_monster['generation']] ).T.to_csv('data/monster.csv')

In [237]:
df_monster.sample()

,jp_name,class,element,weak,generation,name
28,ドスイーオス(Dosuiosu),Bird Wyvern,"Poison, Thunder","Water, Thunder",First,Iodrome


# 4.0. List Itens

In [12]:
url = 'https://monsterhunter.fandom.com/wiki/MHFU:_Item_List'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

page = requests.get( url, headers=headers )
soup = BeautifulSoup( page.text, 'html.parser' )

In [13]:
all_itens = [list( filter( None, p.get_text().split('\n') ) ) for p in soup.find( 'div', class_='mw-parser-output' ).find_all('tr')]

In [14]:
item_data = DataFrame( all_itens )
item_data.columns = ['name', 'description', 'found', 'value', 'rarity']
item_data = item_data[1:]
item_data = item_data.drop_duplicates()
item_data = item_data[~item_data['name'].str.contains('Name')]
item_data = item_data.reset_index( drop=True )

# Create Dataframe for inserv Values
df_ref = DataFrame( index=np.arange( len( item_data ) ), columns=['name_', 'description_', 'found_', 'value_', 'rarity_'] )

# Name
df_ref = pd.concat( [df_ref, item_data['name']], axis=1 )

# Description
df_ref = pd.concat( [df_ref, item_data['description']], axis=1 )

# Found
item_data.loc[item_data['name'] == 'Armor Stone', 'found'] = 'Pokke Farm mining spots and bomb mining or Buy at the item shop for fifteen hundred'
item_found = item_data[~item_data['found'].str.contains('z')]['found']
df_ref = pd.concat( [df_ref, item_found], axis=1 )

# Item Values
item_value2 = item_data['found'].str.extract('(\d+z)')[0]
item_value2.name = 'value'
item_value = item_data[item_data['value'].str.contains('\d+.z')]['value']

item_value = item_value.combine_first( item_value2 )
df_ref = pd.concat( [df_ref, item_value], axis=1 )

# Item Rarity
item_rarity  = item_data[~item_data['value'].str.contains('z')]['value']
item_rarity2 = item_data['rarity']
item_rarity  = item_rarity2.combine_first( item_rarity )
df_ref = pd.concat( [df_ref, item_rarity], axis=1 )

df_ref = df_ref.drop( columns=['name_', 'description_', 'found_', 'value_', 'rarity_'], axis=1 )

# Fill Na Values
# Rarity
df_ref['rarity'] = df_ref['rarity'].fillna('2')

# Values
value_missing = item_data['value'].str.extract( '(^\dz$)' )[0]
df_ref['value'] = df_ref['value'].combine_first( value_missing )
df_ref['value'] = df_ref['value'].fillna( '23z' )

# Found
df_ref.loc[df_ref['name'].str.contains('Tckt'), 'found'] = df_ref.loc[df_ref['name'].str.contains('Tckt'), 'found'].fillna('Gathering Hall')
df_ref.loc[df_ref['name'].str.contains('Pill'), 'found'] = df_ref.loc[df_ref['name'].str.contains('Pill'), 'found'].fillna('Check combine list')
df_ref.loc[df_ref['name'].str.contains('Ancient Potion'), 'found'] = df_ref.loc[df_ref['name'].str.contains('Ancient Potion'), 'found'].fillna('Check combine list')

df_ref['found'] = df_ref['found'].fillna( 'Other Location...' )

# Change Dtypes
df_ref['rarity'] = df_ref['rarity'].astype('int64')

df_ref.to_csv('data/item.csv')

In [15]:
df_ref

,name,description,found,value,rarity
0,Potion,Restores a small amount of health.,Combine Herb and Blue Mushroom or buy it in th...,7z,1
1,Mega Potion,Restores a moderate amount of health.,Combine Potion and Honey,16z,2
2,Nutrients,Increases the maximum Health level slightly.,Combine Godbug and Blue Mushroom,25z,2
3,Mega Nutrients,Medicine that increases the maximum Health lev...,Combine Nutrients and Honey,92z,3
4,Antidote,Effectively removes all traces of poison from ...,Combine Antidote Herb and Blue Mushroom or buy it,6z,1
...,...,...,...,...,...
212,Hunter Soul Tkt,An honor for successful hunters. Symbolizes a ...,Other Location...,0z,5
213,Puppet Ticket,A ticket stamped by two Felyne paws. At your s...,Other Location...,0z,5
214,SpecPuppetTicket,"A Guild-stamped, hand-made ticket. It was made...",Other Location...,0z,5
215,Hypno Ticket,Stamped proof of hunting a Hypnocatrice and fi...,Other Location...,0z,5


# 5.0. Mix Sets

In [129]:
mix_sets = DataFrame( columns = ['weapon', 'head', 'chest', 'hands', 'tasset', 'boots', 'skills'] )

In [ ]:
[['AncientDrgnwoodCan', 'Barrage Piercing', 'Kiaser Vest X', 'Conga Guards Z', 'Kaiser Coat X', 'Genesis Pants Goku', 'Reckless Abandon +3, Capacity Up, Pellet Up'],